Trabalho de Conclusão de Curso - TCC

# MODELO PREDITIVO PARA DETERMINAÇÃO DO PAÍS DE ORIGEM DE MERCADORIAS IMPORTADAS

Diego de Borba Barbosa

Curso de Especialização em Ciência de Dados e Big Data, Puc Minas, abril de 2022

# Coleta de dados

## Importar bibliotecas

In [ ]:
#Carrega as bibliotecas do modelo e algumas utilizadas em testes
#!pip install pydotplus
#!pip install dtreeviz
#from sklearn.inspection import permutation_importance
#import sklearn
#print('The scikit-learn version is {}.'.format(sklearn.__version__))
import functools
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_absolute_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import time
from functools import wraps
import pydotplus 
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import subplots
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 12]

## Parâmetros

In [ ]:
# Define parâmetros para exploração e criação do modelo
#versão do teste com colunas e hiperparâmetros específicos
VERSAO_DO_TESTE = "1.0"
#Anos utilizados para análise exploratória
ANOS = [2019, 2020, 2021]
#Filtro por capítulos de 0 até 99 - filtrar para reduzir consumo de memória
CAPITULO_INICIO = 0
CAPITULO_FIM = 30
#Estrutura y_metricas acumula métricas de avaliação de performance dos modelos
#Deve-se executar seção "Treinamento de modelos" e células posteriores para cada um dos modelos
y_metricas = dict()

## Carga dos datasets

In [ ]:
#Carregamento do dataset
start_time = time.time()

#dataset principal com dados brutos
baseComex = pd.DataFrame()
for ano in ANOS:
    baseComex = baseComex.append(pd.read_csv(".//Dados//IMP_{0}.csv".format(ano), sep = ';', encoding='latin-1'))  

print("Informações do dataset principal IMP_2021")
print("\nTipo: {0}".format(type(baseComex)))
print("Dimensões: {0}".format(baseComex.shape))
print("Campos: {0}".format(baseComex.keys()))
baseComex.describe()

In [ ]:
baseComex.memory_usage().sum()/1024**2
#baseComex.dtypes

In [ ]:
#computar tempo de carregamento
elapsed_time = time.time() - start_time
print("\nLoad Dataset: %.2f" %elapsed_time, "segundos")
baseComex.head()

In [ ]:
#carrega domínios e dataset acessórios
base_pais = pd.read_csv('.//Dados//PAIS.csv', sep = ';' , encoding='latin-1') 
print("Informações do dataset PAIS")
print("\nDimensões: {0}".format(base_pais.shape))
print("Campos: {0}".format(base_pais.keys()))
base_pais.head()

In [ ]:
base_via = pd.read_csv('.//Dados//VIA.csv', sep = ';', encoding='latin-1') 
print("Informações do dataset VIA")
print("\nDimensões: {0}".format(base_via.shape))
print("Campos: {0}".format(base_via.keys()))
base_via

In [ ]:
base_URF = pd.read_csv('.//Dados//URF.csv', sep = ';', encoding='latin-1') 
print("Informações do dataset URF")
print("\nDimensões: {0}".format(base_URF.shape))
print("Campos: {0}".format(base_URF.keys()))
base_URF.head()

In [ ]:
base_NCM = pd.read_csv('.//Dados//NCM.csv', sep = ';', encoding='latin-1')
print("Informações do dataset NCM")
print("\nDimensões: {0}".format(base_NCM.shape))
print("Campos: {0}".format(base_NCM.keys()))
base_NCM.head()

In [ ]:
base_SH6 = pd.read_csv('.//Dados//NCM_SH.csv', sep = ';', encoding='latin-1')
print("Informações do dataset NCM_SH")
print("\nDimensões: {0}".format(base_SH6.shape))
print("Campos: {0}".format(base_SH6.keys()))
base_SH6.head()

## Enriquecimento de dataset

In [ ]:
#dataset enriquecido com domínios e dataset acessórios
baseComexEnriquecida = pd.merge(baseComex, base_pais, on="CO_PAIS", how="inner")
baseComexEnriquecida = pd.merge(baseComexEnriquecida, base_via, on="CO_VIA", how="inner")
baseComexEnriquecida = pd.merge(baseComexEnriquecida, base_URF, on="CO_URF", how="inner")
baseComexEnriquecida = pd.merge(baseComexEnriquecida, base_NCM, on=["CO_NCM","CO_UNID"], how="inner")
baseComexEnriquecida = pd.merge(baseComexEnriquecida, base_SH6, on="CO_SH6", how="inner")

#Filtrar dataset para manter apenas colunas para análise exploratória
lista = ['CO_ANO', 'CO_MES', 'CO_SH2', 'NO_SH2_POR', 'CO_SH4', 'NO_SH4_POR', 'CO_SH6', 
         'CO_NCM', 'CO_UNID', 'CO_PAIS', 'NO_PAIS', 'SG_UF_NCM','CO_VIA','NO_VIA', 
         'CO_URF', 'NO_URF', 'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE', 'VL_SEGURO']

baseComexEnriquecidaAmostra = baseComexEnriquecida[lista]
print("Informações do dataset principal enriquecido para análise exploratória")
#print("\nFiltros por capítulo SH2: {0} a {1}".format(CAPITULO_INICIO, CAPITULO_FIM))
print("Dimensões: {0}".format(baseComexEnriquecidaAmostra.shape))
print("Campos: {0}".format(baseComexEnriquecidaAmostra.keys()))

In [ ]:
#Filtrar dataset para manter apenas colunas para criação do modelo
lista = ['CO_ANO', 'CO_MES', 'CO_SH2', 'CO_SH4', 'CO_NCM', 
         'CO_UNID', 'CO_PAIS', 'SG_UF_NCM','CO_VIA', 'CO_URF', 
         'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB', 'VL_FRETE', 'VL_SEGURO']
baseComexModelo = baseComexEnriquecida[lista]

#Filtrando dataset para selecionar um intervalo de mercadorias por capitulo
baseComexModelo = baseComexModelo.loc[
                                (baseComexModelo['CO_SH2'] >= CAPITULO_INICIO) 
                                & (baseComexModelo['CO_SH2'] <= CAPITULO_FIM)]

#Filtrar dataset para criar modelo - reduzir consumo de memória
#Criar colunas derivadas no dataset do modelo
baseComexModelo["VL_FRETE/KG_LIQUIDO"]= baseComexModelo["VL_FRETE"]/ baseComexModelo["KG_LIQUIDO"]
baseComexModelo["VL_FRETE/KG_LIQUIDO"].replace(np.inf, 0, inplace=True)
baseComexModelo["VL_FRETE/KG_LIQUIDO"].replace(np.nan, 0, inplace=True)

baseComexModelo["VL_FOB/KG_LIQUIDO"]= baseComexModelo["VL_FOB"]/ baseComexModelo["KG_LIQUIDO"]
baseComexModelo["VL_FOB/KG_LIQUIDO"].replace(np.inf, 0, inplace=True)
baseComexModelo["VL_FOB/KG_LIQUIDO"].replace(np.nan, 0, inplace=True)

print("\nInformações do dataset principal para criação do modelo")
print("\nFiltros por capítulo SH2: {0} a {1}".format(CAPITULO_INICIO, CAPITULO_FIM))
print("Dimensões: {0}".format(baseComexModelo.shape))
print("Campos: {0}".format(baseComexModelo.keys()))

In [ ]:
baseComexModelo.head()

# Análise exploratória

## Gráficos

In [ ]:
df_graf_via_fob = baseComexEnriquecidaAmostra.groupby(["NO_VIA"]).agg({"VL_FOB":['sum']})
df_graf_via_kg = baseComexEnriquecidaAmostra.groupby(["NO_VIA"]).agg({"KG_LIQUIDO":['sum']})
df_graf_via_frete = baseComexEnriquecidaAmostra.groupby(["NO_VIA"]).agg({"VL_FRETE":['sum']})
df_graf_via_pais = baseComexEnriquecidaAmostra.groupby(["NO_VIA"]).agg({"NO_PAIS":['nunique']})

#plt.style.use(style ="seaborn")
plt.rcdefaults()

# Cada plot terá o mesmo tamanho de figuras
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,7))

x1 = df_graf_via_fob.index.tolist() 
x2 = df_graf_via_kg.index.tolist() 
x3 = df_graf_via_frete.index.tolist() 
x4 = df_graf_via_pais.index.tolist() 

y1 = df_graf_via_fob['VL_FOB'][ 'sum'].tolist()
y2 = df_graf_via_kg['KG_LIQUIDO']['sum'].tolist()
y3 = df_graf_via_frete['VL_FRETE']['sum'].tolist()
y4 = df_graf_via_pais['NO_PAIS']['nunique'].tolist()

ax1.bar(x1, y1,color='red')
ax2.bar(x2, y2,color='blue')
ax3.bar(x3, y3,color='orange')
ax4.bar(x4, y4,color='green')

ax1.set(title="Valor FOB das mercadorias x Via de transporte", xlabel="Via de transporte", ylabel="Valor FOB")
ax2.set(title="Peso líquido (Kg) das mercadorias x Via de transporte", xlabel="Via de transporte", ylabel="Peso líquido (Kg)")
ax3.set(title="Valor do frete x Via de transporte", xlabel="Via de transporte", ylabel="Valor do frete")
ax4.set(title="Quantidade de países de origem x Via de transporte", xlabel="Via de transporte", ylabel="Quantidade de países")

ax1.set_xticklabels(x1, rotation=45, ha='right')
ax2.set_xticklabels(x2, rotation=45, ha='right')
ax3.set_xticklabels(x3, rotation=45, ha='right')
ax4.set_xticklabels(x4, rotation=45, ha='right')

plt.subplots_adjust(wspace=0.2, hspace=1)
#plt.legend()
plt.show()

In [ ]:
df_graf_pais_fob = baseComexEnriquecidaAmostra.groupby(["NO_PAIS"]).agg({"VL_FOB":['sum']})
df_graf_pais_kg = baseComexEnriquecidaAmostra.groupby(["NO_PAIS"]).agg({"KG_LIQUIDO":['sum']})
df_graf_pais_frete = baseComexEnriquecidaAmostra.groupby(["NO_PAIS"]).agg({"VL_FRETE":['sum']})
df_graf_pais_ncm = baseComexEnriquecidaAmostra.groupby(["NO_PAIS"]).agg({"CO_NCM":['nunique']})

df_graf_pais_fob.columns = ["VL_FOB_sum"]
df_graf_pais_kg.columns = ["KG_LIQUIDO_sum"]
df_graf_pais_frete.columns = ["VL_FRETE_sum"]
df_graf_pais_ncm.columns = ["CO_NCM_nunique"]

top = 15
df_graf_pais_fob = df_graf_pais_fob.sort_values(by="VL_FOB_sum", ascending=False).head(top)
df_graf_pais_kg = df_graf_pais_kg.sort_values(by="KG_LIQUIDO_sum", ascending=False).head(top)
df_graf_pais_frete = df_graf_pais_frete.sort_values(by="VL_FRETE_sum", ascending=False).head(top)
df_graf_pais_ncm = df_graf_pais_ncm.sort_values(by="CO_NCM_nunique", ascending=False).head(top)

plt.rcdefaults()

# Cada plot terá o mesmo tamanho de figuras
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,7))

x1 = df_graf_pais_fob.index.tolist() 
x2 = df_graf_pais_kg.index.tolist() 
x3 = df_graf_pais_frete.index.tolist() 
x4 = df_graf_pais_ncm.index.tolist() 

y1 = df_graf_pais_fob['VL_FOB_sum'].tolist()
y2 = df_graf_pais_kg['KG_LIQUIDO_sum'].tolist()
y3 = df_graf_pais_frete['VL_FRETE_sum'].tolist()
y4 = df_graf_pais_ncm['CO_NCM_nunique'].tolist()

ax1.barh(x1, y1,color='red')
ax2.barh(x2, y2,color='blue')
ax3.barh(x3, y3,color='orange')
ax4.barh(x4, y4,color='green')

ax1.set(title="Países de origem das mercadorias (top {0}) x Valor FOB".format(top), xlabel="Valor FOB", ylabel="Países de origem")
ax2.set(title="Países de origem das mercadorias (top {0}) x Peso líquido (Kg) das mercadorias".format(top), xlabel="Peso líquido (Kg)", ylabel="Países de origem")
ax3.set(title="Países de origem das mercadorias (top {0}) x Valor do frete".format(top), xlabel="Valor do frete", ylabel="Países de origem")
ax4.set(title="Países de origem das mercadorias (top {0})  x Contagem diferentes mercadorias (NCM)".format(top), xlabel="Contagem diferentes mercadorias (NCM)", ylabel="Países de origem")

plt.subplots_adjust(wspace=0.4, hspace=0.5)
#plt.legend()
plt.show()

In [ ]:
#FAZER POR ANO
df_graf_ano_fob = baseComexEnriquecidaAmostra.groupby(["CO_ANO"]).agg({"VL_FOB":['sum']})
df_graf_ano_kg = baseComexEnriquecidaAmostra.groupby(["CO_ANO"]).agg({"KG_LIQUIDO":['sum']})
df_graf_ano_frete = baseComexEnriquecidaAmostra.groupby(["CO_ANO"]).agg({"VL_FRETE":['sum']})
df_graf_ano_ncm = baseComexEnriquecidaAmostra.groupby(["CO_ANO"]).agg({"CO_NCM":['nunique']})

plt.rcdefaults()

# Cada plot terá o mesmo tamanho de figuras
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,7))

x1 = df_graf_ano_fob.index.tolist()
x2 = df_graf_ano_kg.index.tolist() 
x3 = df_graf_ano_frete.index.tolist() 
x4 = df_graf_ano_ncm.index.tolist() 

y1 = df_graf_ano_fob['VL_FOB']['sum'].tolist()
y2 = df_graf_ano_kg['KG_LIQUIDO']['sum'].tolist()
y3 = df_graf_ano_frete['VL_FRETE']['sum'].tolist()
y4 = df_graf_ano_ncm['CO_NCM']['nunique'].tolist()

ax1.plot(x1, y1,color='red')
ax2.plot(x2, y2,color='blue')
ax3.plot(x3, y3,color='orange')
ax4.plot(x4, y4,color='green')

ax1.set(title="Valor FOB x Ano", ylabel="Valor FOB", xlabel="Ano")
ax2.set(title="Peso líquido (Kg) das mercadorias x Ano", ylabel="Peso líquido (Kg)", xlabel="Ano")
ax3.set(title="Valor do frete x Ano", ylabel="Valor do frete", xlabel="Ano")
ax4.set(title="Contagem diferentes mercadorias (NCM) x Ano", ylabel="Contagem diferentes mercadorias (NCM)", xlabel="Ano")

plt.subplots_adjust(wspace=0.2, hspace=0.5)
#plt.legend()
plt.show()

In [ ]:
#FAZER POR MES
df_graf_mes_fob = baseComexEnriquecidaAmostra.groupby(["CO_MES"]).agg({"VL_FOB":['sum']})
df_graf_mes_kg = baseComexEnriquecidaAmostra.groupby(["CO_MES"]).agg({"KG_LIQUIDO":['sum']})
df_graf_mes_frete = baseComexEnriquecidaAmostra.groupby(["CO_MES"]).agg({"VL_FRETE":['sum']})
df_graf_mes_ncm = baseComexEnriquecidaAmostra.groupby(["CO_MES"]).agg({"CO_NCM":['nunique']})

plt.rcdefaults()

# Cada plot terá o mesmo tamanho de figuras
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,7))

x1 = df_graf_mes_fob.index.tolist() 
x2 = df_graf_mes_kg.index.tolist() 
x3 = df_graf_mes_frete.index.tolist() 
x4 = df_graf_mes_ncm.index.tolist() 

y1 = df_graf_mes_fob['VL_FOB'][ 'sum'].tolist()
y2 = df_graf_mes_kg['KG_LIQUIDO']['sum'].tolist()
y3 = df_graf_mes_frete['VL_FRETE']['sum'].tolist()
y4 = df_graf_mes_ncm['CO_NCM']['nunique'].tolist()

ax1.plot(x1, y1,color='red')
ax2.plot(x2, y2,color='blue')
ax3.plot(x3, y3,color='orange')
ax4.plot(x4, y4,color='green')

ax1.set(title="Valor FOB x Mês do ano", ylabel="Valor FOB", xlabel="Mês do ano")
ax2.set(title="Peso líquido (Kg) das mercadorias x Mês do ano", ylabel="Peso líquido (Kg)", xlabel="Mês do ano")
ax3.set(title="Valor do frete x Mês do ano", ylabel="Valor do frete", xlabel="Mês do ano")
ax4.set(title="Contagem diferentes mercadorias (NCM) x Mês do ano", ylabel="Contagem diferentes mercadorias (NCM)", xlabel="Mês do ano")

plt.subplots_adjust(wspace=0.2, hspace=0.5)
#plt.legend()
plt.show()

## Wordcloud

In [ ]:
#CRIA DICT para wordcloud
df_graf_urf_fob = baseComexEnriquecidaAmostra.groupby(["NO_URF"]).agg({"VL_FOB":['sum']})
df_graf_urf_kg = baseComexEnriquecidaAmostra.groupby(["NO_URF"]).agg({"KG_LIQUIDO":['sum']})

df_graf_urf_fob.columns = ["VL_FOB_sum"]
df_graf_urf_kg.columns = ["KG_LIQUIDO_sum"]

df_graf_urf_fob["VL_FOB_sum"].apply(int)
data_urf_fob = df_graf_urf_fob.to_dict('dict')
data_urf_fob = data_urf_fob['VL_FOB_sum']

df_graf_urf_kg["KG_LIQUIDO_sum"].apply(int)
data_urf_kg = df_graf_urf_kg.to_dict('dict')
data_urf_kg = data_urf_kg['KG_LIQUIDO_sum']


In [ ]:
# gerar uma wordcloud
wordcloud = WordCloud(#stopwords=STOPWORDS,
                      background_color="black",
                      width=1600, height=800).generate_from_frequencies(data_urf_fob)

# mostrar a imagem final
fig, ax = plt.subplots(figsize=(20,10))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()

plt.imshow(wordcloud)

In [ ]:
# gerar uma wordcloud
wordcloud = WordCloud(#stopwords=STOPWORDS,
                      background_color="white",
                      width=1600, height=800).generate_from_frequencies(data_urf_kg)
 
# mostrar a imagem final
fig, ax = plt.subplots(figsize=(20,10))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()
plt.imshow(wordcloud)

## Pairplot

In [ ]:
df_pair_plot = baseComexEnriquecidaAmostra.groupby(['CO_SH2', 'CO_PAIS', 'SG_UF_NCM', 'CO_VIA'], as_index=False).agg({'KG_LIQUIDO':['sum'],
       'VL_FOB':['sum'], 'VL_FRETE':['sum']})

df_pair_plot.columns = ['CO_SH2', 'CO_PAIS', 'SG_UF_NCM', 'CO_VIA', 'KG_LIQUIDO_sum', 'VL_FOB_sum', 'VL_FRETE_sum']
df_pair_plot.shape

In [ ]:
#análise
plt.figure(figsize=(36,20))

sns.pairplot(df_pair_plot[['CO_SH2', 'CO_PAIS', 'SG_UF_NCM', 'CO_VIA', 'KG_LIQUIDO_sum',
       'VL_FOB_sum', 'VL_FRETE_sum']], hue="CO_SH2", diag_kind="hist", palette="bright")
plt.show()

In [ ]:
sns.relplot(x='VL_FRETE_sum', y='KG_LIQUIDO_sum', data = df_pair_plot, kind='scatter', size ='VL_FOB_sum', hue ='CO_PAIS', 
            sizes =(20,200), palette = 'RdPu', alpha=.5, height = 7)
plt.show()

In [ ]:
sns.catplot(x='CO_SH2', y='VL_FOB_sum', data = df_pair_plot[['CO_SH2', 'VL_FOB_sum']], kind='swarm')
plt.show()

# Criação do Modelo

## Ajustes dataset (drop e encoder)

In [ ]:
#Conforme análises pode ser necessário desconsiderar essas colunas antes do treinamento
#baseComexModelo.drop("VL_SEGURO", axis=1, inplace=True)
#baseComexModelo.drop("CO_UNID", axis=1, inplace=True)
#baseComexModelo.drop("CO_SH4", axis=1, inplace=True)
#baseComexModelo.drop("CO_SH2", axis=1, inplace=True)
#baseComexModelo.drop("CO_UNID", axis=1, inplace=True)

baseComexModelo.shape
#baseComexModelo.head()

In [ ]:
#encoder da coluna label
le = LabelEncoder()
baseComexModelo["SG_UF_NCM"] = le.fit_transform(baseComexModelo["SG_UF_NCM"])
baseComexModelo["CO_ANO"] = le.fit_transform(baseComexModelo["CO_ANO"])
baseComexModelo["CO_NCM"] = le.fit_transform(baseComexModelo["CO_NCM"])
baseComexModelo["CO_URF"] = le.fit_transform(baseComexModelo["CO_URF"])
baseComexModelo.head()

## Particionamento do dataset

In [ ]:
# Particiona a base de dados
label_target = "CO_PAIS"
X = baseComexModelo.drop(label_target, axis=1)
y = baseComexModelo[label_target]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.15)

In [ ]:
#Quantidade países (target)
unique= np.unique(y)
print('Contagem de target ({0}): {1}'.format(label_target, unique.size))

## Treinamento de modelos

In [ ]:
def print_label_estimador(label):
    print('Classificador: {0}'.format(label))

#inserir label do modelo a treinar e "Executar após"
#DecisionTreeClassifier, RandomForestClassifier, CategoricalNB, KNeighborsClassifier e LogisticRegression
lista_ml = ["DecisionTreeClassifier","RandomForestClassifier", "CategoricalNB", "KNeighborsClassifier", "LogisticRegression"]
label_estimador = lista_ml[0]
print_label_estimador(label_estimador)

### DecisionTreeClassifier

In [ ]:
if label_estimador == "DecisionTreeClassifier":
  ml_estimador = DecisionTreeClassifier(criterion='entropy', 
                                        splitter='best', 
                                        random_state=0, 
                                        min_samples_leaf=1)

  #fit construindo a árvore, nosso modelo
  ml_estimador_model = ml_estimador.fit(X_train, y_train)

### RandomForestClassifier

In [ ]:
if label_estimador == "RandomForestClassifier":
  ml_estimador = RandomForestClassifier(
                            random_state=0,
                            criterion='entropy',
                            max_depth=None,
                            n_estimators=100,
                            min_samples_leaf=1,
                            n_jobs=-1)

  ml_estimador_model = ml_estimador.fit(X_train, y_train)

### Categorical Naive Bayes

In [ ]:
if label_estimador == "CategoricalNB":
  ml_estimador = CategoricalNB(alpha = 0.1, fit_prior = False)
  ml_estimador_model = ml_estimador.fit(X_train, y_train)

### KNeighborsClassifier

In [ ]:
if label_estimador == "KNeighborsClassifier":
  ml_estimador = KNeighborsClassifier(n_neighbors=9, metric= 'hamming')
  ml_estimador_model = ml_estimador.fit(X_train, y_train)  

### LogisticRegression

In [ ]:
if label_estimador == "LogisticRegression":
  ml_estimador = LogisticRegression(random_state=0)
  ml_estimador_model = ml_estimador.fit(X_train, y_train)  

## Avaliação dos modelos

In [ ]:
#predizer y a partir de um X
y_pred = ml_estimador_model.predict(X_test)

In [ ]:
X_train.dtypes

### Acurácia

In [ ]:
# Acurácia
print_label_estimador(label_estimador)
accuracy_test = accuracy_score(y_test, y_pred)
print('Acurácia: {0:.2f}% no treino, {1:.2f}% no teste'.
      format(ml_estimador_model.score(X_train, y_train) * 100, accuracy_test * 100))

### Precisão

In [ ]:
# Precision
print_label_estimador(label_estimador)
precision = precision_score(y_test, y_pred, average='weighted')
print('Precision: %f' % precision)

### Recall

In [ ]:
# Recall
print_label_estimador(label_estimador)
recall = recall_score(y_test, y_pred, average='weighted')
print('Recall: %f' % recall)

### F1-Score

In [ ]:
# F1-Score
print_label_estimador(label_estimador)
f1 = f1_score(y_test, y_pred, average='weighted')
print('F1-Score: %f' % f1)

### Classification report

In [ ]:
print_label_estimador(label_estimador)
print(classification_report(y_test, y_pred))

### Matriz de confusão

In [ ]:
plt.figure(figsize=(20,20))
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_table = pd.DataFrame(data=cnf_matrix)
data = {
    'Ocorreu': y_test,
    'Predito': y_pred
}
df = pd.DataFrame(data, columns=['Ocorreu','Predito'])
conf = pd.crosstab(df['Ocorreu'], df['Predito'], rownames=['Ocorreu'], colnames=['Predito'])
sns.heatmap(conf, annot=True, annot_kws={"size":5}, cmap=plt.cm.Blues)

plt.title('Matriz de Confusão')
plt.show()

### Feature Importance

In [ ]:
print_label_estimador(label_estimador)

try:
    if label_estimador == "CategoricalNB" or label_estimador == "KNeighborsClassifier" or label_estimador == "LogisticRegression" or label_estimador == "SupportVectorMachine":
        #feature_importances_ para Naive Bayes
        imps = permutation_importance(ml_estimador_model, X_test, y_test)
        importancia = pd.DataFrame({"Feature":X.columns.values, "Importância (%)": 100 * imps.importances_mean})  
    else:
        #feature para Tree
        importancia = pd.DataFrame({"Coluna":X.columns.values, "Importância (%)": 100 * ml_estimador_model.feature_importances_})
except BaseException:
    print("feature_importances_ indisponível na versão sklearn.__version__")
    
importancia.sort_values(by=["Importância (%)"], ascending=False)

### Validação cruzada

In [ ]:
#executar Cross Validation com 5 partes

cv = KFold(n_splits = 5, shuffle = True)
scores = cross_val_score(ml_estimador, X_train, y_train, cv=cv, scoring="accuracy" )

In [ ]:
print_label_estimador(label_estimador)

scores
print(scores.mean())
print(scores.std())
mean_accuracy = scores.mean()
dv = scores.std()
print('Acurácia média: {:.2f}%'.format(mean_accuracy*100))
print('Intervalo de acurácia: [{:.2f}% ~ {:.2f}%]'
           .format((mean_accuracy - 2*dv)*100, (mean_accuracy + 2*dv)*100))

### GridSearchCV

In [ ]:
print_label_estimador(label_estimador)

list_min_samples_leaf = list(range(1, 7, 1))

k_list = list(range(1,21))
    
if label_estimador == "CategoricalNB":
  param_grid = {'var_smoothing': np.logspace(0,-9, num=30)}

elif label_estimador == "KNeighborsClassifier":
  param_grid = {'n_neighbors': k_list,
                'metric': ['minkowski','canberra', 'hamming', 'euclidean', 'manhattan']}
elif label_estimador == "LogisticRegression":
  param_grid = {'C': np.logspace(-5, 8, 5), 'penalty': ['l1', 'l2']}
else:
  param_grid = {"criterion": ['entropy', 'gini'],
                "min_samples_leaf": list_min_samples_leaf}

In [ ]:
print_label_estimador(label_estimador)
grid_search = GridSearchCV(ml_estimador_model, param_grid, scoring="accuracy", cv=5)
grid_search.fit(X_train, y_train)

classifier_rf = grid_search.best_estimator_ 
print("Melhores parametros {} com o valor de acurácia {} ".
      format(grid_search.best_params_, grid_search.best_score_))

### RandomizedSearchCV

In [ ]:
print_label_estimador(label_estimador)

randomized_search = RandomizedSearchCV(ml_estimador_model, param_grid, scoring="accuracy", cv=5, n_iter=10,)
randomized_search.fit(X_train, y_train)

randomized_search.best_estimator_ 
randomized_search.best_params_, randomized_search.best_score_
print("Melhores parametros {} com o valor de acurácia {} ".
      format(randomized_search.best_params_, randomized_search.best_score_))

### Erro médio absoluto

In [ ]:
print_label_estimador(label_estimador)

mean = mean_absolute_error(y_test, y_pred)
print(mean)

### Visualizar árvore

In [ ]:
### Teste visualização pequenas árvores
    if label_estimador == "DecisionTreeClassifier - false":
        name_classes = [str(val) for val in ml_estimador_model.classes_]
        # Create DOT data
        dot_data = tree.export_graphviz(ml_estimador_model, out_file=None, 
                                        #proportion=True,
                                        rounded = True,
                                        filled = True, 
                                        feature_names = ml_estimador_model.feature_names_in_,  
                                        class_names = name_classes)
        # Draw graph
        graph = pydotplus.graph_from_dot_data(dot_data)  
        # Show graph
        Image(graph.create_png())

### Avaliação comparativa de performance

In [ ]:
#Estrutura y_metricas acumula métricas de avaliação de performance dos modelos
#Deve-se executar após seção "Treinamento de modelos" para cada um dos modelos
y_metricas[label_estimador] = [accuracy_test, mean_accuracy, precision, recall, f1]
print(y_metricas)

In [ ]:
#Criar gráfico comparativo
x = np.arange(5) 
plt.figure(figsize=(12,7))
width = 0.1

for i in range(0, len(lista_ml)):
  if lista_ml[i] in y_metricas.keys():
    plt.bar(x +(width*(i)), y_metricas[lista_ml[i]], width, label=lista_ml[i]) 

plt.xticks(x, ["Acurácia", "Acurácia_CV", "Precisão", "Recall", "F1-Score"])
plt.legend()
plt.title('Avaliação comparativa de performance')
plt.show()